# Importing Libraries

In [ ]:
import torch
from tqdm.notebook import tqdm

from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

from transformers import AdamW, get_linear_schedule_with_warmup

from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

import random
import pickle

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm

# Reading data and analysing

In [ ]:
batch_size = 2
test_batch_size = 1
epochs = 2

In [ ]:
train_df = pd.read_csv('../input/text-classification-dataset/hindi/hindi/hindi_train.csv',lineterminator='\n')
dev_df = pd.read_csv('../input/text-classification-dataset/hindi/hindi/hindi_valid.csv',lineterminator='\n')
test_df = pd.read_csv('../input/text-classification-dataset/hindi/hindi/hindi_test.csv',lineterminator='\n')

In [ ]:
train_df = train_df.drop(columns=['Unnamed: 0'])
dev_df = dev_df.drop(columns=['Unnamed: 0'])
test_df = test_df.drop(columns=['Unnamed: 0'])

In [ ]:
train_df.columns = ['text', 'label']
dev_df.columns = ['text', 'label']
test_df.columns = ['text', 'label']

In [ ]:
train_df['label'].value_counts()

In [ ]:
train_df.head()

In [ ]:
dev_df['label'].value_counts()

In [ ]:
dev_df.head()

In [ ]:
test_df['label'].value_counts()

In [ ]:
test_df.head()

# Label encoding

In [ ]:
train_labels = train_df.label.unique()

label_dict = {}
for index, label in enumerate(train_labels):
    label_dict[label] = index
label_dict

In [ ]:
with open('./label_encoding.pkl', 'wb') as fp:
    pickle.dump(label_dict, fp)

In [ ]:
train_df['label_encode'] = train_df.label.replace(label_dict)
dev_df['label_encode'] = dev_df.label.replace(label_dict)
test_df['label_encode'] = test_df.label.replace(label_dict)

In [ ]:
train_df.head()

In [ ]:
dev_df.head()

In [ ]:
test_df.head()

In [ ]:
print("Train samples: ", train_df.shape)
print("Valid samples: ", dev_df.shape)
print("Test samples: ", test_df.shape)

# Model Architecture

## Tokenizing dataset

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased', do_lower_case=False)

In [ ]:
encoded_data_train = tokenizer.batch_encode_plus(
    train_df.text.values,
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=512,
    return_tensors='pt'
)

In [ ]:
encoded_data_val = tokenizer.batch_encode_plus(
    dev_df.text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=512, 
    return_tensors='pt'
)

In [ ]:
encoded_data_test = tokenizer.batch_encode_plus(
    test_df.text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=512, 
    return_tensors='pt'
)

In [ ]:
tokenizer.save_pretrained('./tokenizer/')

## Creating input_ids, attention_masks and labels

In [ ]:
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(train_df.label_encode.values)

In [ ]:
print(input_ids_train[0])
print(attention_masks_train[0])
print(labels_train[0])

In [ ]:
input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(dev_df.label_encode.values)

In [ ]:
print(input_ids_val[0])
print(attention_masks_val[0])
print(labels_val[0])

In [ ]:
input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']
labels_test = torch.tensor(test_df.label_encode.values)

In [ ]:
print(input_ids_test[0])
print(attention_masks_test[0])
print(labels_test[0])

## Preparing the data with Data generator

In [ ]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)
dataset_test = TensorDataset(input_ids_test, attention_masks_test, labels_test)

In [ ]:
print(dataset_train[0])
print(dataset_val[0])
print(dataset_test[0])

## Model

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

## Preparing data generator with Data Loader

In [ ]:
dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

In [ ]:
dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [ ]:
dataloader_test = DataLoader(dataset_test, 
                                   sampler=SequentialSampler(dataset_test), 
                                   batch_size=test_batch_size)

## Adding the AdamW optimizer and scheduler

In [ ]:
optimizer = AdamW(model.parameters(), lr=1e-5, eps=1e-8)

In [ ]:
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

## Functions to compute f1_score and accuracy_per_class

In [ ]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

In [ ]:
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

## Setting the seed

In [ ]:
seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

## Model train and evaluate methods

In [ ]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in tqdm(dataloader_val):
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [ ]:
for epoch in range(1, epochs+1):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'./finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

# Model Testing

In [ ]:
test_loss, predictions, true_vals = evaluate(dataloader_test)

## Model f1_score

In [ ]:
test_f1 = f1_score_func(predictions, true_vals)
tqdm.write(f'Validation loss: {test_loss}')
tqdm.write(f'F1 Score (Weighted): {test_f1}')

## Model accuracy per class score

In [ ]:
accuracy_per_class(predictions, true_vals)

## Model classification report

In [ ]:
classification_report(true_vals, predictions, target_names=train_labels)

In [ ]:
classification_result = []

label_dict_inverse = {v: k for k, v in label_dict.items()}
preds_flat = np.argmax(predictions, axis=1).flatten()
labels_flat = true_vals.flatten()

for i in range(len(labels_flat)):
    actual_label = labels_flat[i]
    actual_label = label_dict_inverse[actual_label]
    sent = test_df['text'][i]
    predicted_label = label_dict_inverse[preds_flat[i]]
    
    classification_result.append({'Actual': actual_label, 'predict': predicted_label, 'sentence': sent})

In [ ]:
with open('classification_results.pkl', 'wb') as fp:
    pickle.dump(classification_result, fp)